In [ ]:
import warnings
import pandas as pd
# , Range1d#, LabelSet
from bokeh.models import ColumnDataSource, Label, Text, Span, HoverTool, Div
from bokeh.layouts import column, row, gridplot, layout
from dmyplant2 import (
    cred, MyPlant, Engine,
    FSMOperator, startstopFSM, FSM_splot, FSM_splotBC, FSM_VLine, FSM_add_Notations, FSM_add_Alarms, FSM_add_Warnings,
    bokeh_show, dbokeh_chart, add_dbokeh_vlines, get_cycle_data2, disp_result, disp_alarms, disp_warnings,
    cvset, cplotdef, equal_adjust, count_columns, load_data, get_cycle_data, get_cycle_data2, figures)
import dmyplant2
import time
import scipy.fftpack
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import arrow
import ipywidgets as widgets
from ipywidgets import AppLayout, Button, Layout, VBox, HBox, Label, HTML, interact, IntSlider
from IPython.display import HTML, display
display(HTML("<style>.container {width:94% !important;}</style>"))
from pprint import pprint as pp
from datetime import datetime
import numpy as np
from bokeh.plotting import figure, output_file, show
import bokeh
# default warn => SettingWithCopyWarning
pd.options.mode.chained_assignment = None
warnings.simplefilter(action='ignore', category=UserWarning)

cred()
mp = MyPlant(3600)
Engine._list_cached_validations();
#mp._fetch_installed_base(); # refresh local installed fleet database

In [ ]:
# #lookup = 'BMW Landshut 4.10'
# #lookup = 'GRR'
# #lookup = 'V441'
lookup = 'Forsa Hartmoor'
# #lookup = 'Textile'
#lookup = 'CREYKE'
motor_num = 1

In [ ]:
def sfun(x):
    #return all([ (lookup in str(x['Design Number'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    return all([ (lookup in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
fleet = mp.search_installed_fleet(sfun).drop('index', axis=1)
fleet = fleet.sort_values(by = "Engine ID",ascending=True).reset_index(drop='index')

In [ ]:
#fleet.T;

In [ ]:
motor = fleet.iloc[motor_num]
modes = ['undefined','OFF','MAN','AUTO']
success = [True,False]
#success = [True]
e=Engine.from_fleet(mp,motor)
pp_from=e['Commissioning Date']
#pp_from='2019-11-25'
#pp_to='2019-12-01'
pp_to=datetime.now()
motor['IB Site Name'] + ' ' + motor['Engine ID'], modes, success

In [13]:
#Run State Machines
fsm = FSMOperator(e, p_from=pp_from, p_to=pp_to)
fsm.run0(enforce=True, silent=False, debug=False)
fsm.run1(silent=False, successtime=300, debug=False) # run Finite State Machine

FSM1: 100%|██████████████████████| 14305/14305 [00:01<00:00, 7281.94 messages/s]


In [14]:
fsm.run2(silent = False, debug = True)
fsm.run4(silent = False, debug = True)

FSM4: 100%|████████████████████████████| 250/250 [00:00<00:00, 6291.67 starts/s]


In [15]:
fsm.results

{'sn': '1486152',
 'save_date': None,
 'first_message': Timestamp('2022-01-10 00:37:14.999000064'),
 'last_message': Timestamp('2022-05-15 23:44:16.328999936'),
 'starts': [{'run2': True,
   'run4': True,
   'no': 0,
   'success': 'undefined',
   'mode': 'undefined',
   'starttime': Timestamp('2022-01-10 07:19:55.152999936'),
   'endtime': Timestamp('2022-01-10 07:22:17.180999936'),
   'cumstarttime': 142.027999,
   'oilfilling': 124.371,
   'degasing': 17.656999,
   'starter': nan,
   'speedup': nan,
   'idle': nan,
   'synchronize': nan,
   'loadramp': nan,
   'targetoperation': nan,
   'rampdown': nan,
   'coolrun': nan,
   'runout': nan,
   'startstoptiming': {'oilfilling': [{'start': Timestamp('2022-01-10 07:19:55.152999936'),
      'end': Timestamp('2022-01-10 07:21:59.524000')}],
    'degasing': [{'start': Timestamp('2022-01-10 07:21:59.524000'),
      'end': Timestamp('2022-01-10 07:22:17.180999936')}]},
   'alarms': [],
   'warnings': [],
   'targetload': nan,
   'ramprate': n

In [16]:
#fsm.save_results('test.dfsm')

In [17]:
fsm = FSMOperator.load_results(mp, 'test.dfsm')
fsm.results['info']

{'save_date': Timestamp('2022-05-09 00:00:00'),
 'p_from': Timestamp('2022-01-10 00:00:00'),
 'p_to': datetime.date(2022, 5, 9),
 'run2': True,
 'starts': 230,
 'Name': 'Forsa Hartmoor M02',
 'DesignNumber': 'AL87',
 'EngineType': '624',
 'EngineVersion': 'H12',
 'serialNumber': '1486152',
 'id': 159397}

In [18]:
modes = ['undefined','OFF','MAN','AUTO']
success = [True,False]
#success = [True]
rdf = fsm.starts
rdf

,run2,no,success,mode,starttime,endtime,cumstarttime,startpreparation,starter,speedup,...,TJ_GasPress1_at_Min,TJ_GasTemp1_at_Min,TJ_Pos_at_Min,rpm_dmax,rpm_dmin,rpm_spread,Lambda_rpm_max,TempOil_rpm_max,TempCoolWat_rpm_max,targetoperation_org
0,True,0,undefined,undefined,2022-01-10 07:19:55.152999936,2022-01-10 07:23:06.796999936,142.027998,98.271000,3.628999,22.471000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,True,1,undefined,undefined,2022-01-10 11:03:58.406000128,2022-01-10 11:07:57.500999936,194.530998,93.909999,3.836000,20.272000,...,NaN,NaN,NaN,1508.000,1506.000000,2.000000,1.538000,45.269800,61.2,NaN
2,True,2,undefined,undefined,2022-01-10 11:08:28.972999936,2022-01-10 11:08:33.212000000,4.239000,4.239000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,True,3,undefined,MAN,2022-01-10 11:09:18.824000000,2022-01-10 11:10:56.833999872,98.009999,98.009999,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,True,4,undefined,AUTO,2022-01-10 11:10:58.550000128,2022-01-10 11:14:08.936999936,144.624997,60.638999,4.139000,18.257999,...,1061.000000,29.700,4.000000,1514.000,1511.000000,3.000000,1.561000,54.300000,63.8,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,True,225,success,AUTO,2022-05-07 10:05:08.887000064,2022-05-07 11:13:15.783000064,278.112998,104.224000,3.634999,18.558000,...,1046.110000,22.589,56.545701,1528.313,1499.875000,28.438000,1.539092,72.077099,75.8,3621.220999
226,True,226,success,AUTO,2022-05-07 15:35:09.334000128,2022-05-07 21:43:12.824000000,278.665997,100.209999,4.135000,18.959000,...,1043.799000,28.200,62.269800,1523.469,1502.578575,20.890425,1.490989,57.385001,60.9,21621.382000
227,True,227,success,AUTO,2022-05-08 04:04:07.382000128,2022-05-08 08:43:14.907000064,277.617998,99.969999,4.142000,19.474999,...,1044.001000,26.900,67.200000,1520.750,1504.518000,16.232000,1.477000,53.223800,57.7,16285.766000
228,True,228,success,AUTO,2022-05-08 16:05:08.044000000,2022-05-08 17:13:17.487000064,278.955998,99.602000,4.136000,19.866999,...,1041.483000,29.500,67.172500,1517.710,1505.577640,12.132360,1.475000,52.592899,59.4,3623.230000


In [19]:
rda = rdf[:].reset_index(drop='index')
rda = rda[(rda['mode'].isin(modes) & rda['success'].isin(success))].reset_index(drop='index')
rdb = rda
#rda[filterFSM.run2filter_content].round(2)
print(f"Starts: {rdf.shape[0]}, Successful: {rdf[rdf['success'] == True].shape[0]}, Failed: {rdf[rdf['success'] == False].shape[0]} => {rdf[rdf['success'] == True].shape[0]/rdf.shape[0]*100.0:3.1f}%")
pd.DataFrame.from_dict(fsm._e.dash, orient='index').T

Starts: 230, Successful: 0, Failed: 0 => 0.0%


,Name,Engine ID,Design Number,Engine Type,Engine Version,P,P_nom,BMEP,serialNumber,id,Count_OpHour,val start,oph@start,oph parts,LOC
0,Forsa Hartmoor M02,M02,AL87,624,H12,24,4495.0,24.5,1486152,159397,488.0,2022-01-10,54,434.0,0.1355


In [20]:
#rde = rda[(rda.starttime > fsm._e['Commissioning Date']) & (rda['success']) & (rda.targetload > 2800.0)].copy()
#rde = rda[(rda.starttime > fsm._e['Commissioning Date']) & (rda.count_alarms > 0)].copy()
#rde = rda[(rda.starttime > fsm._e['Commissioning Date']) & (rda['success'])].copy()
rde = rda
rde['datetime'] = pd.to_datetime(rde['starttime'])
rde['isuccess'] = rde.apply(lambda x: 1 if x['success'] else 0, axis=1)
vec = ['startpreparation','speedup','idle','synchronize','loadramp','targetload','ramprate','cumstarttime','targetoperation','rampdown','coolrun','runout','isuccess']
display(rde[vec].describe().round(2))
dfigsize = (18,12)
dset = [
    {'col':['cumstarttime'],'_ylim':(-600,800), 'color':'darkblue'},
    {'col':['runout'],'_ylim':(0,100) },
    {'col':['targetload'],'_ylim':(-4000,26000) },
    {'col':['ramprate'],'_ylim':(-5,7)},
    {'col':['loadramp'],'_ylim':(-150,900), 'color':'red'},
    {'col':['speedup'],'_ylim':(-100,200), 'color':'orange'},
    {'col':['synchronize'],'_ylim':(-20,400)},
    {'col':['startpreparation'],'_ylim':(-1000,800)},
    {'col':['count_warnings','count_alarms','isuccess'],'_ylim':(-1,200), 'color':['rgba(255,165,0,0.3)','rgba(255,0,0,0.3)','rgba(0,128,0,0.2)'] },
    {'col':['no'],'_ylim':(0,1000), 'color':['rgba(0,0,0,0.1)'] },
    #{'col':['count_warnings','count_alarms','no'],'ylim':(-1,200), 'color':['rgba(255,165,0,0.3)','rgba(255,0,0,0.3)','rgba(0,0,0,0.1)'] }
    ]
dset = equal_adjust(dset, rde, do_not_adjust=[-1])
ftitle = f"{fsm._e}"
fig = dbokeh_chart(rde, dset, style='both', figsize=dfigsize ,title=ftitle);
bokeh_show(fig)

ValueError: Expected a 1D array, got an array with shape (0, 49)

In [ ]:
def f(x=0):
    global vv; vv = x; startversuch = rdb.iloc[x]
    #ftitle = f"{fsm._e} ----- Start {startversuch['no']} {startversuch['mode']} | {'SUCCESS' if startversuch['success'] else 'FAILED'} | {startversuch['starttime'].round('S')} CumStart: {startversuch['cumstarttime']:0.1f}"
    #display(HTML(ftitle));
    disp_result(startversuch)
interact(f, x = IntSlider(0, 0, rdb.shape[0]-1 , 1, layout=widgets.Layout(width='90%')));

In [ ]:
# PLotter
def myfigures(e):
    return {
    'actors' : [
    {'col':['Power_PowerAct'], 'ylim':(0,5000), 'color':'red', 'unit':'kW'},
    {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
    {'col':['Ignition_ITPAvg'],'ylim': [-10, 30], 'color':'rgba(255,0,255,0.4)', 'unit':'°KW'},
    {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
    {'col':['Various_Values_PosThrottle','Various_Values_PosTurboBypass'],'ylim': [-10, 110], 'color':['rgba(105,105,105,0.6)','rgba(165,42,42,0.4)'], 'unit':'%'},
    ],
    'lubrication' : [
    {'col':['Power_PowerAct'], 'ylim':(0,5000), 'color':'red', 'unit':'kW'},
    {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
    {'col':['Hyd_PressCrankCase'],'ylim': [-100, 100], 'color':'orange', 'unit':'mbar'},
    {'col':['Hyd_PressOilDif'],'ylim': [0, 2], 'color':'black', 'unit': 'bar'},
    {'col':['Hyd_PressOil'],'ylim': [0, 10], 'color':'brown', 'unit': 'bar'},
    {'col':['Hyd_TempOil','Hyd_TempCoolWat','Hyd_TempWatRetCoolOut'],'ylim': [0, 110], 'color':['#2171b5','orangered','hotpink'], 'unit':'°C'},
    ],
    'exhaust' : [
    {'col':['Power_PowerAct'], 'ylim':(0,5000), 'color':'red', 'unit':'kW'},
    {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
    {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
    {'col':e.dataItemsCyl('Exhaust_TempCyl*'),'ylim': [400, 700], 'unit':'°C'},
    {'col':e.dataItemsCyl('Knock_Valve_Noise_Cyl*'),'ylim': [0, 4000], 'unit':'V'},
    ],
    'ignition' : [
    {'col':['Power_PowerAct'], 'ylim':(0,5000), 'color':'red', 'unit':'kW'},
    {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
    {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
    {'col':e.dataItemsCyl('Monic_VoltCyl*'),'ylim': [0, 100], 'unit':'kV'},
    {'col':e.dataItemsCyl('Ignition_ITPCyl*'),'ylim': [0, 40], 'unit':'°KW'},
    {'col':e.dataItemsCyl('Knock_KLS98_IntKnock_Cyl*'),'ylim': [-30, 60], 'unit':'%'},
    ],
        
}
lfigures = myfigures(fsm._e)
plotdef, vset = cplotdef(mp, lfigures)
#dmaxlength = 1800
dmaxlength = None
dminlength = None
startversuch = rdb.iloc[vv]
ftitle = f"{fsm._e} ----- Start {startversuch['no']} {startversuch['mode']} | {'SUCCESS' if startversuch['success'] else 'FAILED'} | {startversuch['starttime'].round('S')}"
data = get_cycle_data2(fsm, startversuch, max_length=dmaxlength, min_length=dminlength, cycletime=1, silent=True, p_data=vset)
data['power_diff'] = pd.Series(np.gradient(data['Power_PowerAct']))
for doplot in plotdef:
    dset = lfigures[doplot]
    ltitle = f"{ftitle} | {doplot}"
    try:
        if count_columns(dset) > 12: # no legend, if too many lines.
            fig = FSM_splot(fsm, startversuch, data, dset, title=ltitle, legend=False, figsize=(18, 10))
        else:
            fig = FSM_splot(fsm, startversuch, data, dset, title=ltitle, figsize=(18, 10))

        fig = FSM_add_Notations(fig, fsm, startversuch)
        disp_alarms(startversuch)
        disp_warnings(startversuch)
        fig = FSM_add_Alarms(fig, fsm, startversuch)
        fig = FSM_add_Warnings(fig, fsm, startversuch)
        bokeh_show(fig)
    except Exception as err:
        print(err)

In [ ]:
for i, v in enumerate(fsm.runlogdetail(startversuch, statechanges_only=True)):
    print(f"{i:3} {v}")

In [ ]:
for i, v in enumerate(fsm.runlogdetail(startversuch, statechanges_only=False)):
    print(f"{i:3} {v}")

In [ ]:
rdb = rda
vec = ['startpreparation','speedup','idle','synchronize','loadramp','targetload','ramprate','cumstarttime','targetoperation','rampdown','coolrun','runout']
display(_=rdb[vec].hist(bins=30,layout=(4,4),figsize=(20,20)))
#ax_list[0][2].set_xlim((0,10))
#display(rdb[vec].hist(bins=30,figsize=(20,20)))
display(rdb[vec].describe().round(2))

In [ ]:
import time
import dmyplant2
from bokeh.models import ColumnDataSource, Label, Text, Span, HoverTool #, Range1d#, LabelSet

dset2 = [{'col':['Power_PowerAct','helpline'], 'ylim':(-1000,12000), 'color':['red','rgba(0,128,0,0.2)'], 'unit':'kW'},
         {'col':['power_diff','power_diff_help'], '_ylim':(0,5000), 'color':['rgba(255,0,0,0.2)','rgba(0,128,0,0.2)'], 'unit':'kW/s'}]
ratedload = fsm._e['Power_PowerNominal']
t0 = time.time()
for i, startversuch in rdb[-3:].iterrows() : 
#for i, startversuch in tqdm(rdb.iterrows(), total=rdb.shape[0], ncols=80, mininterval=1, unit=' starts', desc="FSM Run2"):
    data, xmax, ymax, duration, ramprate = dmyplant2.loadramp_edge_detect(fsm,startversuch)
    if not data.empty:
        print(f"Start: {startversuch['no']:3d} xmax: {xmax}, ymax: {ymax:6.0f}, duration: {duration:5.1f}, ramprate: {ramprate / ratedload * 100.0:4.2f} %/s")
        data['power_diff'] = pd.Series(np.gradient(data['Power_PowerAct']))
        #data['power_diff_help'] = pd.Series(np.gradient(data['helpline']))
        fig = FSM_splot(fsm, startversuch, data, dset2, figsize=(14,6))
        fig.add_layout(Span(location=0.0,dimension='width',x_range_name='default', y_range_name='0',line_color='black', line_dash='solid', line_alpha=0.4)) 
        fig.add_layout(Span(location=ymax,dimension='width',x_range_name='default', y_range_name='0',line_color='blueviolet', line_dash='dashdot', line_alpha=0.4, line_width=2)) 
        fig.add_layout(Span(location=xmax,dimension='height',line_color='blueviolet', line_dash='dashdot', line_alpha=0.4, line_width=2)) 
        fig = FSM_add_Notations(fig, fsm, startversuch)
        #fig = FSM_add_Alarms(fig, fsm, startversuch)
        #fig = FSM_add_Warnings(fig, fsm, startversuch)
        bokeh_show(fig)
    else:
        print(f"Start: {startversuch['no']:3d} no data, no improvement possible.")
t1 = time.time()
print(f"{(t1-t0):4.1f} sec")    

In [ ]:
mfn = e._fname + '_messages.txt'
#fsm.save_messages(mfn)
print(mfn)